In [1]:
import os
import torch
import torchvision

from pytorch_lightning.loggers.neptune import NeptuneLogger

from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint

from model import PlaceholderModel
from pathlib import Path
from argparse import Namespace


from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.managed_loop import optimize
from ax.utils.notebook.plotting import render, init_notebook_plotting

from ax.service.ax_client import AxClient

from env import NEPTUNE_API_TOKEN

In [2]:
algo = "unet3d"
architecture = "arch_1"

# data_path = '../../data/Proca_fiducial_scaled_cropped.hdf5'
data_path = '/scratch/ns4486/numerical-relativity-interpolation/Proca_fiducial_scaled_cropped.hdf5'

# checkpoint_path = '/Users/nikhilvs/checkpoints'
checkpoint_path = f'/scratch/prs392/capstone/checkpoints/{algo}/{architecture}'
experiment_name = 'unet3d_batch1'

d = os.path.join(checkpoint_path, experiment_name)
Path(d).mkdir(parents=True, exist_ok=True)
versions = [o for o in os.listdir(d) if os.path.isdir(os.path.join(d,o))]

versions = sorted(versions)

In [3]:
checkpoint_path

'/scratch/prs392/capstone/checkpoints/unet3d/arch_1'

In [4]:
from tensorboard.backend.event_processing import event_accumulator
import numpy as np

def _load_run(path):
    event_acc = event_accumulator.EventAccumulator(path)
    event_acc.Reload()
    data = {}

    for tag in sorted(event_acc.Tags()["scalars"]):
        x, y = [], []

        for scalar_event in event_acc.Scalars(tag):
            x.append(scalar_event.step)
            y.append(scalar_event.value)

        data[tag] = (np.asarray(x), np.asarray(y))
    return data

In [5]:
from pytorch_lightning.core.saving import load_hparams_from_yaml

list_existing_hparams = []
list_of_val_loss = []

for version in versions:
    hparam_path = os.path.join(checkpoint_path, experiment_name, version, 'hparams.yaml')
    hparams_new = load_hparams_from_yaml(hparam_path)

    print(hparams_new)
#     print(min(_load_run(os.path.join(checkpoint_path, experiment_name, version))['val_loss'][1]))

#     list_existing_hparams.append(hparams_new)
#     list_of_val_loss.append(min(_load_run(os.path.join(checkpoint_path, experiment_name, version))['val_loss'][1]))

{}
{}
{}
{}


/ext3/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:37: RuntimeWarning:

Missing Tags: /scratch/prs392/capstone/checkpoints/unet3d/arch_1/unet3d_batch1/NUM-84/hparams.yaml.

/ext3/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:37: RuntimeWarning:

Missing Tags: /scratch/prs392/capstone/checkpoints/unet3d/arch_1/unet3d_batch1/NUM-85/hparams.yaml.

/ext3/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:37: RuntimeWarning:

Missing Tags: /scratch/prs392/capstone/checkpoints/unet3d/arch_1/unet3d_batch1/NUM-86/hparams.yaml.

/ext3/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:37: RuntimeWarning:

Missing Tags: /scratch/prs392/capstone/checkpoints/unet3d/arch_1/unet3d_batch1/NUM-87/hparams.yaml.



In [6]:
def train_evaluate(parameterization):
    torch.cuda.empty_cache()
    
    seed_everything(123)

    print(parameterization)
    
    hparams = Namespace(**parameterization)
    model = PlaceholderModel(hparams, data_path)
#     logger = TensorBoardLogger(checkpoint_path, name=experiment_name)
    neptune_logger = NeptuneLogger(
        api_key=NEPTUNE_API_TOKEN,
        project_name="nyuds2019/numerical-relativity-interpolation",
        experiment_name=experiment_name,  # Optional,
        params=parameterization,  # Optional,
        tags=["Param" ,"UNet3D", "L1", "3D"]  # Optional,
    )
    checkpoint_callback = ModelCheckpoint(
        filepath=None,
        save_top_k=5,
        save_last = True,
        verbose=False,
        monitor='val_loss',
        mode='min',
        prefix=''
    )
    trainer = Trainer(
#          logger = logger, 
         logger = neptune_logger,
         default_root_dir=checkpoint_path,
         checkpoint_callback = checkpoint_callback,
         max_epochs = 200,
         gpus = -1,
         distributed_backend = 'dp',
#          val_check_interval=0.10,
         log_save_interval=1,
         row_log_interval=1,
         fast_dev_run=False
    )
    trainer.fit(model)
    
    model_best_loss = model.best_loss
    
    del trainer, model, checkpoint_callback, neptune_logger, hparams
    
    torch.cuda.empty_cache()
    
    return {
        'loss': (model_best_loss, 0.0)
    }

In [7]:
ax_client = AxClient()
ax_client.create_experiment(
    name = experiment_name,
    parameters=[
        {"name": "batch_size", "type": "choice", "values": [16, 20]},
        {"name": "lr", "type": "range", "bounds": [1e-4, 0.1], "log_scale": True},
        {"name": "lr_type", "type": "choice", "values": ['adam', 'sgd']},
        {"name": "scheduler_epoch", "type": "range", "bounds": [5, 20], "value_type": "int"},
        {"name": "scheduler_step_size", "type": "range", "bounds": [0.1, 1.0]}
    ],
    objective_name="loss",
    minimize=True,
)

[INFO 10-14 23:33:42] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 2 decimal points.
[INFO 10-14 23:33:42] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.


In [ ]:
total_number_of_trials = 50

fixed_params = [
    {
        'batch_size': 20,
        'lr': 0.001,
        'lr_type': 'adam',
        'scheduler_epoch': 5,
        'scheduler_step_size': 0.5
    }
]

for params in fixed_params:
    parameters, trial_index = ax_client.attach_trial(params)
    ax_client.complete_trial(trial_index=trial_index, raw_data=train_evaluate(parameters))
    total_number_of_trials -= 1
    
for _ in range(total_number_of_trials):
    parameters, trial_index = ax_client.get_next_trial()
    print(trial_index)
    ax_client.complete_trial(trial_index=trial_index, raw_data=train_evaluate(parameters))

[INFO 10-14 23:33:43] ax.service.ax_client: Attached custom parameterization {'batch_size': 20, 'lr': 0.0, 'lr_type': 'adam', 'scheduler_epoch': 5, 'scheduler_step_size': 0.5} as trial 0.


{'batch_size': 20, 'lr': 0.001, 'lr_type': 'adam', 'scheduler_epoch': 5, 'scheduler_step_size': 0.5}
https://ui.neptune.ai/nyuds2019/numerical-relativity-interpolation/e/NUM-88


NeptuneLogger will work in online mode
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name      | Type   | Params
-------------------------------------
0 | model     | UNet3D | 1 M   
1 | criterion | L1Loss | 0     


Traceback (most recent call last):
  File "/ext3/miniconda3/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/ext3/miniconda3/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/ext3/miniconda3/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/ext3/miniconda3/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


Saving latest checkpoint..


...

[INFO 10-15 10:32:55] ax.service.ax_client: Completed trial 5 with data: {'loss': (tensor(0.0615, device='cuda:0'), 0.0)}.
[INFO 10-15 10:32:55] ax.service.ax_client: Generated new trial 6 with parameters {'lr': 0.0, 'scheduler_epoch': 14, 'scheduler_step_size': 0.36, 'batch_size': 20, 'lr_type': 'sgd'}.


6
{'lr': 0.0034207762604701914, 'scheduler_epoch': 14, 'scheduler_step_size': 0.35754756368696694, 'batch_size': 20, 'lr_type': 'sgd'}
https://ui.neptune.ai/nyuds2019/numerical-relativity-interpolation/e/NUM-97


NeptuneLogger will work in online mode
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name      | Type   | Params
-------------------------------------
0 | model     | UNet3D | 1 M   
1 | criterion | L1Loss | 0     


Traceback (most recent call last):
  File "/ext3/miniconda3/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/ext3/miniconda3/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/ext3/miniconda3/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/ext3/miniconda3/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
from UNet import UNet3D
from torchsummary import summary

In [ ]:
model = UNet3D(2, 1)

In [ ]:
summary(model.cuda(), (2, 72, 72, 72), )

In [ ]:
from loader import SingleChannelDataset

In [ ]:
data_path = '/scratch/ns4486/numerical-relativity-interpolation/Proca_fiducial_scaled_cropped.hdf5'

dataset = SingleChannelDataset(data_path)

In [ ]:
torch.cat([dataset[0][0].unsqueeze(0), dataset[0][1].unsqueeze(0)], dim=1).shape

In [ ]:
dataset[0][2].shape